In [2]:
import os
import asyncio
import time
from dotenv import load_dotenv
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.google_genai import GoogleGenAI
from duckduckgo_search.exceptions import RatelimitException
 
# ✅ Load API keys
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY") or os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY or GOOGLE_API_KEY not set.")
 
# ✅ Google Gemini LLM
llm = GoogleGenAI(
    model="gemini-1.5-flash",
    api_key=api_key,
    temperature=0.3,
)
 
# ✅ Custom system prompt
formatter_prompt = """
You are a mobile specifications expert.
When given a phone model, return:
 
**Specifications:**
- Display: ...
- Chipset: ...
- Battery: ...
- OS: ...
- Camera: ...
- Storage: ...
- Other: ... (e.g., RAM, Build, Release Year, Charging)
 
**Expert Summary:**
• One or two concise sentences highlighting performance, build, or unique features.
"""
 
# ✅ Rate-limit safe wrapper for DuckDuckGo
def safe_duckduckgo_search(query, max_results=5, retries=3, delay=2):
    tool_spec = DuckDuckGoSearchToolSpec()
    for attempt in range(retries):
        try:
            return tool_spec.duckduckgo_full_search(query, max_results=max_results)
        except RatelimitException:
            if attempt < retries - 1:
                wait_time = delay * (2 ** attempt)
                print(f"⚠️ Rate limit hit. Retrying in {wait_time}s...")
                time.sleep(wait_time)
            else:
                print("❌ DuckDuckGo rate limit reached. No results available.")
                return []
 
# ✅ Main async function
async def main():
    tool_spec = DuckDuckGoSearchToolSpec()
    tools = tool_spec.to_tool_list()
 
    workflow = AgentWorkflow.from_tools_or_functions(
        tools_or_functions=tools,
        llm=llm,
        system_prompt=formatter_prompt,
        verbose=False,  # keep output clean
    )
 
    phone_query = "Realme 8 specifications"
 
    # 🔹 Use safe search
    results = safe_duckduckgo_search(phone_query, max_results=5)
    if results:
        print(f"🔎 Got {len(results)} search results for '{phone_query}'")
 
    # 🔹 Get structured response
    response = await workflow.run(user_msg=phone_query)
 
    print("\n📱 Final Answer:\n")
    print(str(response))
 
# ✅ Run in Jupyter (no asyncio.run)
await main()

⚠️ Rate limit hit. Retrying in 2s...
⚠️ Rate limit hit. Retrying in 4s...
❌ DuckDuckGo rate limit reached. No results available.

📱 Final Answer:

I am sorry, I cannot find the specifications for the Realme 8 using the available tools.  I need access to a database or API with phone specifications to answer your query.

